In [1]:
import tensorflow as tf


print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


2024-11-13 18:53:51.032409: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-13 18:53:51.059460: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-13 18:53:51.059606: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
with tf.device('/GPU:0'):
    a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
    b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
    c = tf.matmul(a, b)
    print(c)

2024-11-13 18:54:02.509041: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-13 18:54:02.511187: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-13 18:54:02.511653: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-13 18:54:02.512021: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so retur

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


2024-11-13 18:54:03.934628: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [3]:
import tensorflow as tf

USE_GPU = True

if USE_GPU and tf.config.list_physical_devices('GPU'):
    device = 'GPU'
    print("Using GPU")
else:
    device = 'CPU'
    print("Using CPU")

Using GPU


In [5]:
import tensorflow_datasets as tfds

# Load the ImageNet dataset

data_dir = '/home/asko/tensorflow_datasets/downloads/manual'

# Load the ImageNet Resized dataset (e.g., 32x32)
dataset, info = tfds.load('imagenet_resized/64x64', data_dir=data_dir, with_info=True, as_supervised=True)

# Split the dataset into training and validation sets
train_dataset = dataset['train']
val_dataset = dataset['validation']

# Print the dataset info
print(info)



tfds.core.DatasetInfo(
    name='imagenet_resized',
    full_name='imagenet_resized/64x64/0.1.0',
    description="""
    This dataset consists of the ImageNet dataset resized to fixed size. The images
    here are the ones provided by Chrabaszcz et. al. using the box resize method.
    
    For [downsampled ImageNet](http://image-net.org/download.php) for unsupervised
    learning see `downsampled_imagenet`.
    
    those from the other ImageNet datasets provided by Tensorflow datasets. See the
    original
    [label list](https://github.com/PatrykChrabaszcz/Imagenet32_Scripts/blob/master/map_clsloc.txt),
    and the
    [labels used by this dataset](https://github.com/tensorflow/datasets/blob/master/tensorflow_datasets/image_classification/imagenet_resized_labels.txt).
    Additionally, the original authors 1 index there labels which we convert to 0
    indexed by subtracting one.
    """,
    config_description="""
    Images resized to 64x64
    """,
    homepage='https://patrykc

In [9]:
from keras.applications import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np

# Load the VGG16 model pre-trained on ImageNet
model = VGG16(weights='imagenet')


In [ ]:

from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array

# Load an example image and preprocess it
img_path = '/home/asko/Documents/workspace/Fall-24/682/project/elephant.jpg'  # Replace with the path to your image
img = load_img(img_path, target_size=(224, 224))
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# Run the image through the model to obtain predictions
predictions = model.predict(x)
print(predictions.shape)
print('Predicted:', decode_predictions(predictions, top=3))


In [6]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications.vgg16 import preprocess_input

data_dir = '/home/asko/tensorflow_datasets/downloads/manual'
dataset, info = tfds.load('imagenet_resized/64x64', data_dir=data_dir, with_info=True, as_supervised=True)

train_dataset = dataset['train']
val_dataset = dataset['validation']


def preprocess(image, label):
    image = tf.image.resize(image, [224, 224])
    image = preprocess_input(image) 
    return image, label

batch_size = 32
train_dataset = (train_dataset
                 .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
                 .shuffle(1000)  
                 .batch(batch_size)
                 .prefetch(tf.data.AUTOTUNE))

val_dataset = (val_dataset
               .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
               .batch(batch_size)
               .prefetch(tf.data.AUTOTUNE))

In [ ]:
train_dataset

In [ ]:
import torch
import torch.nn.functional as F  # useful stateless functions

dtype = torch.float32
print_every = 100

def train_part34(model, optimizer, epochs=1):
    
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(train_dataset):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            
            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()
            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                print()

In [11]:
tf.device('/GPU:0' if device == 'GPU' else '/CPU:0')

import pandas as pd

output_file = 'predictions.csv'

# Create CSV file and write headers if it doesn’t exist
with open(output_file, mode='w') as f:
    f.write("Prediction\n")

for batch_idx, (images, labels) in enumerate(train_dataset):
    print(batch_idx, images.shape, labels.shape)
    predictions = model.predict(images)
    batch_df = pd.DataFrame(predictions, columns=[f'Prediction_{i}' for i in range(predictions.shape[1])])
    batch_df.to_csv(output_file, mode='a', header=False, index=False)
    if batch_idx >= 499:
        break
    

0 (32, 224, 224, 3) (32,)
1/1 [==============================] - 2s 2s/step
1 (32, 224, 224, 3) (32,)
1/1 [==============================] - 2s 2s/step
2 (32, 224, 224, 3) (32,)
1/1 [==============================] - 2s 2s/step
3 (32, 224, 224, 3) (32,)
1/1 [==============================] - 2s 2s/step
4 (32, 224, 224, 3) (32,)
1/1 [==============================] - 2s 2s/step
5 (32, 224, 224, 3) (32,)
1/1 [==============================] - 2s 2s/step
6 (32, 224, 224, 3) (32,)
1/1 [==============================] - 2s 2s/step
7 (32, 224, 224, 3) (32,)
1/1 [==============================] - 2s 2s/step
8 (32, 224, 224, 3) (32,)
1/1 [==============================] - 2s 2s/step
9 (32, 224, 224, 3) (32,)
1/1 [==============================] - 2s 2s/step
10 (32, 224, 224, 3) (32,)
1/1 [==============================] - 2s 2s/step
11 (32, 224, 224, 3) (32,)
1/1 [==============================] - 2s 2s/step
12 (32, 224, 224, 3) (32,)
1/1 [==============================] - 2s 2s/step
13 (32, 2

KeyboardInterrupt: 

In [ ]:
predictions.shape